In [ ]:
!python --version

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

In [ ]:
!python --version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/bert4rec_project/

In [ ]:
%ls

In [ ]:
!pip install -r requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
import tensorflow as tf

from dataclasses import dataclass, asdict

## Load parameters

In [14]:
@dataclass
class Config:
    num_layers:int=2
    num_heads:int=8
    d_model:int=128
    dropout:float=0.1
    layer_norm_eps:float=1e-12
    activation:str="gelu"
    vocab_size:int=40857
    max_seq_len:int=256
    learning_rate:float=1e-4
    batch_size:float=64

config=Config()

# Prepare data

In [7]:
from bert4rec.dataprocessor import DataLoader

In [8]:
filenames = glob.glob('../data/processed/*')
filenames

['../data/processed/test.tfrecords',
 '../data/processed/training.tfrecords',
 '../data/processed/vocab.pkl']

In [9]:
train_dataloader = DataLoader(
    input_file_path=filenames[1],
    batch_size=config.batch_size,
    max_seq_length=config.max_seq_len,
)

train_data = train_dataloader.to_tf_dataset()

2022-11-24 14:54:15.047938: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
test_dataloader = DataLoader(
    input_file_path=filenames[0],
    batch_size=config.batch_size,
    max_seq_length=config.max_seq_len,
)

test_data = train_dataloader.to_tf_dataset()

## Train model

In [12]:
from bert4rec.bert import BertModel
from bert4rec.trainer import BertTrainer

In [15]:
model = BertModel(
    num_layers=config.num_layers,
    num_heads=config.num_heads,
    d_model=config.d_model,
    dropout=config.dropout,
    layer_norm_eps=config.layer_norm_eps,
    activation=config.activation,
    vocab_size=config.vocab_size,
    max_seq_len=config.max_seq_len,
)
model.build((config.batch_size, config.max_seq_len))
model.summary()

Model: "bert_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 positional_embedding_1 (Pos  multiple                 5262592   
 itionalEmbedding)                                               
                                                                 
 transformer_encoder_layer_1  multiple                 198528    
 2 (TransformerEncoderLayer)                                     
                                                                 
 transformer_encoder_layer_1  multiple                 198528    
 3 (TransformerEncoderLayer)                                     
                                                                 
 dropout_15 (Dropout)        multiple                  0         
                                                                 
 dense_30 (Dense)            multiple                  16512     
                                                      

In [16]:
bert_trainer = BertTrainer(model)
bert_trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [18]:
bert_trainer.fit(train_data, batch_size=config.batch_size, epochs=100)

Epoch 1/100
   1/2142 [..............................] - ETA: 29:15:59 - loss: 9.5650 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
tf.test.gpu_device_name()

In [ ]:
bert_trainer.fit(train_ds, batch_size=config.batch_size, epochs=5)

In [ ]:
tf.saved_model.save(bert_trainer, "../models/bert4rec_movies")

## Test model